In [1]:
#引用套件
import boto3
import os
import json
import decimal
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
# Get the service resource.
dynamodb = boto3.resource('dynamodb',region_name=os.environ["region_name"],endpoint_url="http://cc104.dynamodb.local:8000",aws_access_key_id=os.environ["aws_access_key_id"],aws_secret_access_key=os.environ["aws_secret_access_key"])

print("ok")

# 建立dynamodb的資料表
try:
    table = dynamodb.create_table(
    TableName='Movies',
    KeySchema=[
        {
            'AttributeName': 'year',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'title',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'year',
            'AttributeType': 'N'
        },
        {
            'AttributeName': 'title',
            'AttributeType': 'S'
        },

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 2,
        'WriteCapacityUnits': 2
    }
)
    print(table,"資料表:", table.table_status)
except Exception as err:    
    table = dynamodb.Table('Movies')
    print(table,"資料表:", table.table_status)

#print("讀取資料庫:",table.item_count,"項 物件")


print("將movies.json資料導入movie3資料表中:\n================================")
with open("movies.json") as json_file:
    movies = json.load(json_file, parse_float = decimal.Decimal)
    for movie in movies:
        year = int(movie['year'])
        title = movie['title']
        info = movie['info']
        table.put_item(
           Item={
               'year': year,
               'title': title,
               'info': json.dumps(str(info))
            }
        )
        print(year,title,"....新增成功")
print("================================")


class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if abs(o) % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)



print("=============使用put方法新增資料記錄在Movies table內=============")

table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

response = table.put_item(
   Item={
        'year': year,
        'title': title,
        'info': {
            'plot':"Nothing happens at all.",
            'rating': decimal.Decimal(0)
        }
    }
)

print("PutItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

print("=============使用get方法=============")
title = "The Big New Movie"
year = 2015

try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    print("GetItem succeeded:")
    print(json.dumps(item, indent=4, cls=DecimalEncoder))
    

    
print("=================更新項目============")    
title = "The Big New Movie"
year = 2015    

    
response = table.update_item(
    Key={
        'year': year,
        'title': title
    },
    UpdateExpression="set info.rating = :r, info.plot=:p, info.actors=:a",
    ExpressionAttributeValues={
        ':r': decimal.Decimal(5.5),
        ':p': "Everything happens all at once.",
        ':a': ["Larry", "Moe", "Curly"]
    },
    ReturnValues="UPDATED_NEW"
)

print("UpdateItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

ok
dynamodb.Table(name='Movies') 資料表: ACTIVE
將movies.json資料導入movie3資料表中:
2013 Rush ....新增成功
2013 Prisoners ....新增成功
2013 The Hunger Games: Catching Fire ....新增成功
2013 Thor: The Dark World ....新增成功
2013 This Is the End ....新增成功
=============使用put方法新增資料記錄在Movies table內=============
PutItem succeeded:


NameError: name 'DecimalEncoder' is not defined